<a href="https://colab.research.google.com/github/Rajeeb321123/Deep-learning-tensorFlow-Journey/blob/master/08_introduction_to_NLP_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamental in TensorFLow

NLP has the goal of deriving out of natural language ( could be sequence text or speech)

Another  common term for NLP problems is sequence problem (seq2seq).

## Check GPU

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Get the helper function

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper function for the notebook

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-04-06 13:16:54--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-04-06 13:16:54 (95.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

original source of dataset: https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip the data
unzip_data("nlp_getting_started.zip")

--2024-04-06 13:17:01--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.126.207, 74.125.201.207, 74.125.69.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.126.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-04-06 13:17:01 (119 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To Visualize our text samples, we first have to read them in, one way to do so would be to use Python: https://realpython.com/read-write-files-python/

But I perfer get visual straight away.

So anther way to do this use pandas... (may not work for large dataset exceeding RAM: here 16 GB).

for loading large text dataest: https://www.tensorflow.org/tutorials/load_data/text


In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head(n=10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [5]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # 1 means 100% of data, 0.1 means 10% of data
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
# How many example s of each class are there?
# for knowing our dataset is balanced or not?
train_df.target.value_counts()

#60 and 40 split between 0 and 1 : pretty balance (50-50)

target
0    4342
1    3271
Name: count, dtype: int64

Here we work with almost balance dataset but for imbalance dataset ( no near 50-50 split) refer to: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

In [10]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [11]:
a = train_df_shuffled["target"] # 1d
print(a.values, a.values.shape) # array([1, 0, 1, ..., 0, 1, 1])

a = train_df_shuffled[["target"]] # 2d
print(a.values, a.values.shape) # array([[1], [0], [1], ..., [1]])

a = train_df_shuffled[["text","target"]] # 2d
print(a.values[0], a.values.shape)

[1 0 1 ... 0 1 1] (7613,)
[[1]
 [0]
 [1]
 ...
 [0]
 [1]
 [1]] (7613, 1)
['So you have a new weapon that can cause un-imaginable destruction.' 1] (7613, 2)


In [12]:
# a little tutorial on comprehension
a = [1, 2, 3, 4, 5]

b = [ x if x < 3 else "" for x in a]
print(b)
b = [ x  for x in a if x < 3]
print(b)

[1, 2, '', '', '']
[1, 2]


In [13]:
# Let's visualize some random training dataset
import random
random_index = random.randint(0, len(train_df)-5) # create a random index not higher than total no of samples
print(f"random_index: {random_index}")
#itertuples turing dataframe values to tuple
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples(): # random index eg 170 to 175, five values
  _, text, target = row # skipping index
  print(f"Target: {target}", "(real disaster)" if target > 0 else "{not real disaster}")
  print(f"Text:\n{text}\n")
  print("---\n")

random_index: 3081
Target: 0 {not real disaster}
Text:
Do you feel like you are sinking in unhappiness? Take the quiz: http://t.co/OrJb3j803F http://t.co/MWdHXYfrag

---

Target: 1 (real disaster)
Text:
Why are you deluged with low self-image? Take the quiz: http://t.co/XsPqdOrIqj http://t.co/CQYvFR4UCy

---

Target: 0 {not real disaster}
Text:
I come downstairs trying to look as normal as possible in front of my mom and Rhiannon screams 'HUNGOVER???? Huh???' Uhmmmm no ????????

---

Target: 0 {not real disaster}
Text:
@tremblayeh we like big hail and we cannot lie!! #SirMixAlot

---

Target: 1 (real disaster)
Text:
The famine is coming to an end #Bluebell http://t.co/p9ryMfjcUX

---



### Split data into training and validation sets

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
# Use train_test_split to split  to split training and validation set

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation,
                                                                            random_state=42)

In [16]:
# Check the lengths
len(train_sentences),len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [17]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text to numbers

* Tokenization vs Embedding video no: 228 watch full video -> must

When dealing with a text problem, one of the first things we'll have to do before we can build a model is to convert our text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number.
* `Embedding` - create a matrix of feature vector fo each each token (the size of feature vector can be defined and `weights can be learned`).

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [19]:
# use the default TextVectorization parameters
# This is only the default bu we will modify later here.
#max_token=10000 means only 10000 most common word
text_vectorizer = TextVectorization(max_tokens=None, # How many words in our vocabulary: None: no limit ( automaically add <00V>)
                                    standardize = 'lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # n-words:join splitted words before tokenizaion. None:each word, each token
                                    output_mode='int', # how to map token to token. here int but can be specific function like tf_idf
                                    #How long do we want our sequence to be : 50 ,100, None..
                                    #automatially set sequence to longest sequence or tweet lenght. add zero ad end for short tweets
                                    output_sequence_length=None, # Batch need to same length. Eg here Each Tweet length are different
                                    # aadd zero ad end for short tweets to match (output_sequence_length: none: means longest one)
                                    pad_to_max_tokens=False #if true reult in shape (Batch_size, Max_tokesn)
                                    )

In [20]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [21]:
len(train_sentences[0].split())

7

In [22]:
# Find the average number of tokens (words) in the training tweets for keeping our model small
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

# seem no of token or words is 15

15

In [23]:
# Setup text Vectorization variable
max_vocab_length = 10000 # max number of words  to have in our vocabulary
# 15 from round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))
max_length= 15 # max length our sequence will be (e.g. how many words from a tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length = max_length,
                                    pad_to_max_tokens=True,
                                    )

In [24]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [25]:
# Create a sample sentences and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [26]:
# Choose  a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 RSS: Judge orders Texas to recognize spouse on same-sex death certificate  http://t.co/TZIolfTe5i      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[4708, 2412, 1268,  758,    5, 2754, 8195,   11, 4688,  154, 6007,
           1,    0,    0,    0]])>

In [27]:
# How unique words from vocabulary we found in our train_sentence
# if Max_token: None as hyperparameter : automatically set no of words_in_vocab to each Unique word
# if Max_token: 10000 like we did : words_in_vocab will be 100000
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our train_data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words

print(f"Number of words in vocab in train_data: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

#[UNK] means unkown: 10000 words may be small but its ok here

Number of words in vocab in train_data: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating a Embedding using an Embedding layer

To make our embedding, we're going to use TensorFlow's embeddings layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameter we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary.
* `output_dim` = the size our output embedding vector, for example, a value of 100 would mean each token get represented by a vector 100 long.
* `input_length` = length of the sequences being passed to the embedding layer.


In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length, # set the input shape,
                              output_dim = 128, # Neural network especially that using GPU works greate with number divisible by 8 eg. 128
                              input_length = max_length, # how long is each input
                              )
embedding

In [29]:
# for visualizaion only on random sentence

# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 SCREAMS AT MY OWN MOTHER http://t.co/gBEpdi0WzT      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0155344 , -0.01744924, -0.02347206, ...,  0.0089469 ,
          0.03864891, -0.02672762],
        [-0.04017633,  0.00449901, -0.03459548, ...,  0.04552053,
          0.02118986, -0.02819904],
        [ 0.01299078, -0.00220354,  0.02698361, ...,  0.01980032,
          0.02114035, -0.01854401],
        ...,
        [-0.00180399, -0.04783306, -0.00303447, ...,  0.02747837,
          0.04452762,  0.04723983],
        [-0.00180399, -0.04783306, -0.00303447, ...,  0.02747837,
          0.04452762,  0.04723983],
        [-0.00180399, -0.04783306, -0.00303447, ...,  0.02747837,
          0.04452762,  0.04723983]]], dtype=float32)>

In [30]:
# Check out a single token's embedding for the first word of our sentence e.g. Worked
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.0155344 , -0.01744924, -0.02347206,  0.02312047, -0.04463908,
        -0.04085923, -0.04494007, -0.04792819,  0.02093229,  0.036495  ,
        -0.04589136, -0.04736853, -0.0262243 ,  0.02910418, -0.01092893,
        -0.00027404, -0.01287488,  0.04071293,  0.023094  ,  0.00176934,
         0.04725958,  0.00951304,  0.00797872,  0.01115301, -0.01138232,
        -0.01720474,  0.03626886,  0.00028964,  0.01961089, -0.04934642,
        -0.02749857,  0.00624392, -0.03572725,  0.00628542, -0.03690308,
        -0.02800119, -0.01453046, -0.00719764,  0.04488924,  0.02434171,
         0.01202104,  0.04736232,  0.03436725, -0.02398779, -0.03285148,
         0.00803728, -0.00277298, -0.00571905, -0.03332342, -0.04919815,
        -0.03104075,  0.03079199,  0.04811808, -0.02771536, -0.0296526 ,
        -0.02349046,  0.01887104,  0.04805036,  0.040582  , -0.01237217,
         0.00333568,  0.01337408,  0.00247902, -0.00899804,  0.02524897,
  

## Modelling a text dataset (running a series of experiments)

Now we've a way to turn our text sequence into number, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from sklearn ML map: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidriection-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* MOdel 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* MOdel 7: Same as model 6 with 10% of training data.

How are we going to approach all of these?

Use the standard steps in modeling in tensorflow.

* Create a model
* Build a model
* Fit the model
* Evaluate model
* Improve the model through experimentaion
* Save and reload the model

### Model 0: Getting a baseline

As with all machine Learning modelling experiments, it's important to create a baseline model so  we've got a benchmark for future experiments to build upon.

To create a baseline, we'll use Sklearn's Multinomial Naive Bayes using TF-IDF to convert our words to numbers.

> 🔑 It's common practice to use non-DL algorithm as a baseline of their speed and later using DL to see if we can improve upon them.

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer #Convert a collection of raw documents to a matrix of TF-IDF features or vectors.
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling Pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # Convert words to number using tfidf
    ("clf", MultinomialNB()) # model the text
])

# Fit teh pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [32]:
# Evaluate our  baseline model # as we use classification (multinomialNB) our defalut metrics for evaluation is accuracy
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [33]:
train_df.target.value_counts()
# as our classification is nearly 50 - 50 split , so baseline( guessing ) is aruond 50% or 55% and our accuracy is better than guessing at 80%

target
0    4342
1    3271
Name: count, dtype: int64

In [34]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Createing an evaluation function for our model experiments

We could evaluate all our model's predictions with different matrix every time, this can be easily fixed with function.

Let's create one to compare our model's predictions with the truth labels using the functions.
* Accuracy
* Precision
* Recall
* F1-score

Deep overview for many model evaluation use: sklearn: https://scikit-learn.org/stable/modules/model_evaluation.html

In [38]:
# Function to evalute: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculate model accuracy, precision, recall and f1 score of a binary classification model.
  """

  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall, f1-score using average = weighted average # weighted average is good for slight model imbalance like here
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")

  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}

  return model_results

In [39]:
# Get baseline results
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

In [136]:
# It is imp for creating tensorboard callback if we are expreimenting with different model
# Create a tensorboard callback (need to create a new one of each model)
from helper_functions import  create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [142]:
# Build a model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # input are 1-dimensional strings # making sure our inputs are string

x = text_vectorizer(inputs) # turn the input text int0 numbers # Text vectorization layer
x = embedding(x) # our embedding layer # create an embedding of the numberized inputs

#if we donot use global average pooling we get 1 ouput per owrd of shape(None, 15, 1)  but we want (None, 1) or 1 for each sentence
# Here global average pooling 1d is must. # 1D is for sequential data, where as 2D is for ima
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector. This will improve the accuracy.
# we could have use Flatten()(x) but this will cause overfitting as Flatten donot remove parameter like GlobalAveragePooling

# x = layers.GloablMaxPool1D()(x) # we can also add this

outputs = layers.Dense(1, activation="sigmoid")(x) # our output layer 1 only as disaster or not disaster (binary classification)

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

# For sequential model
# model = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(1,), dtype=tf.string),
#     text_vectorizer,
#     embedding ,
#     layers.GlobalAveragePooling1D()
#     tf.keras.layers.Dense(1, activation="sigmoid")
# ])

In [143]:
# Wrong one without Global average pooling 1D
# model_1.summary()
# after textvectorization each semtemce is turn into 15 token: each token: 1 word in numeric form
# after embedding we embed each token into 128 numerical value( each 0 to 1)
# Model: "model_1_dense"
# _________________________________________________________________
#  Layer (type)                Output Shape              Param #
# =================================================================
#  input_29 (InputLayer)       [(None, 1)]               0

#  text_vectorization_1 (Text  (None, 15)                0
#  Vectorization)

#  embedding (Embedding)       (None, 15, 128)           1280000

#  dense_38 (Dense)            (None, 15, 1)             129

# =================================================================
# Total params: 1280129 (4.88 MB)
# Trainable params: 1280129 (4.88 MB)
# Non-trainable params: 0 (0.00 Byte)



# Right one with Global average pooling 1D
model_1.summary()
# Very imp there was 128 per each(of the 15) token,
# After Global Average Pooling 128 values per each whole sentence( all 15 token )

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_30 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense_39 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [144]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [145]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels), # combining them into tuple
                              callbacks = [create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                       experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240406-145939
Epoch 1/5
215/215 [==============================] - 7s 29ms/step - loss: 0.5620 - accuracy: 0.7710 - val_loss: 0.5197 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 6s 30ms/step - loss: 0.3893 - accuracy: 0.8597 - val_loss: 0.4663 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 7s 34ms/step - loss: 0.3069 - accuracy: 0.8857 - val_loss: 0.4607 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 5s 25ms/step - loss: 0.2556 - accuracy: 0.9051 - val_loss: 0.4687 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 6s 29ms/step - loss: 0.2177 - accuracy: 0.9216 - val_loss: 0.4841 - val_accuracy: 0.7822


In [146]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4841 - accuracy: 0.7822


[0.48410698771476746, 0.7821522355079651]

In [147]:
# Make some preidictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [148]:
model_1_pred_probs[0]

array([0.4055189], dtype=float32)

In [149]:
# look at the first 10 prediction
model_1_pred_probs[:10]

array([[0.4055189 ],
       [0.79647136],
       [0.99685615],
       [0.14509824],
       [0.08795652],
       [0.93934244],
       [0.89941245],
       [0.99446976],
       [0.9753374 ],
       [0.26630035]], dtype=float32)

In [152]:
# Convert model prediction probabilities to label format
#round will change below .5 to 0 and above 0.5 to 1
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) #Removes dimensions of size 1 from the shape of a tensor.
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [154]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true = val_labels,
                                    y_pred = model_1_preds )

model_1_results

{'accuracy': 78.21522309711287,
 'precision': 0.7846540517195822,
 'recall': 0.7821522309711286,
 'f1': 0.7799245444538409}

In [155]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [157]:
import numpy as np
# changing dictionary to list
# seems like our baseline better the model_1 we just created upon baseline
np.array(list(model_1_results.values()) > np.array(list(baseline_results.values())))

array([False, False, False, False])